# Description

**TODO**: should probably be moved to preprocessing folder.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

# import numpy as np
import pandas as pd

import conf

# from multiplier import MultiplierProjection
from entity import Gene

# from data.cache import read_data
# from data.hdf5 import simplify_trait_fullcode, HDF5_FILE_PATTERN

# Settings

In [3]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses')

In [4]:
OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
display(OUTPUT_DATA_DIR)
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data')

In [5]:
OUTPUT_RAW_DATA_DIR = Path(OUTPUT_DATA_DIR, "raw")
display(OUTPUT_RAW_DATA_DIR)
OUTPUT_RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data/raw')

In [6]:
OUTPUT_PROJ_DATA_DIR = Path(OUTPUT_DATA_DIR, "proj")
display(OUTPUT_PROJ_DATA_DIR)
OUTPUT_PROJ_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data/proj')

# Load LINCS consensi drugbank (Daniel)

In [7]:
# TODO: hardcoded
input_file = Path(
    conf.DATA_DIR, "hetionet", "lincs-v2.0", "consensi-drugbank.tsv.bz2"
).resolve()

display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/hetionet/lincs-v2.0/consensi-drugbank.tsv.bz2')

In [8]:
lincs_data = pd.read_csv(input_file, sep="\t", index_col="perturbagen").T

In [9]:
lincs_data.shape

(7467, 1170)

In [10]:
lincs_data.head()

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
100,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
1000,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
10000,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
10001,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
10005,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


In [11]:
assert lincs_data.index.is_unique

In [12]:
# drubback ids are consistent
_tmp = lincs_data.columns.map(len).unique()
assert _tmp.shape[0] == 1

In [13]:
assert lincs_data.columns.is_unique

## Gene IDs to Gene names

In [14]:
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

pandas2ri.activate()

In [15]:
clusterProfiler = importr("clusterProfiler")

In [16]:
# lincs_data = lincs_data.rename(columns=GENE_ENTREZ_ID_TO_SYMBOL).rename(columns=GENE_SYMBOL_TO_ENSEMBL_ID).T

In [17]:
# _gene_id_len = lincs_data.index.map(len)

In [18]:
# _not_mapped_genes = lincs_data[_gene_id_len != 15].index.copy()

In [19]:
# _not_mapped_genes

In [20]:
_now_mapped_genes = clusterProfiler.bitr(
    lincs_data.index.tolist(),
    fromType="ENTREZID",
    toType="ENSEMBL",
    OrgDb="org.Hs.eg.db",
)

R[write to console]: Loading required package: org.Hs.eg.db



R[write to console]: Loading required package: AnnotationDbi



R[write to console]: Loading required package: stats4



R[write to console]: Loading required package: BiocGenerics



R[write to console]: Loading required package: parallel



...

R[write to console]: Loading required package: S4Vectors



R[write to console]: 
Attaching package: ‘S4Vectors’




R[write to console]: The following object is masked from ‘package:base’:

    expand.grid




R[write to console]: 



R[write to console]: 'select()' returned 1:many mapping between keys and columns



In [21]:
_now_mapped_genes.shape

(8259, 2)

In [22]:
# some genes have entrez ids that map to several ensembl id
display(_now_mapped_genes[_now_mapped_genes["ENTREZID"].duplicated(keep=False)])

,ENTREZID,ENSEMBL
3,10000,ENSG00000117020
4,10000,ENSG00000275199
12,100129361,ENSG00000256537
13,100129361,ENSG00000262986
14,100129361,ENSG00000281930
...,...,...
8202,9859,ENSG00000276725
8243,9920,ENSG00000176595
8244,9920,ENSG00000273645
8288,9994,ENSG00000118412


In [23]:
_now_mapped_genes = _now_mapped_genes.assign(
    in_phenomexcan=_now_mapped_genes["ENSEMBL"].apply(
        lambda x: x in Gene.GENE_ID_TO_NAME_MAP
    )
)

In [24]:
_now_mapped_genes[_now_mapped_genes["in_phenomexcan"]].shape

(7125, 3)

In [25]:
_now_mapped_genes.head()

,ENTREZID,ENSEMBL,in_phenomexcan
1,100,ENSG00000196839,True
2,1000,ENSG00000170558,True
3,10000,ENSG00000117020,True
4,10000,ENSG00000275199,False
5,10001,ENSG00000133997,True


In [26]:
_now_mapped_genes = _now_mapped_genes[
    _now_mapped_genes["in_phenomexcan"]
].drop_duplicates(subset=["ENTREZID"])

In [27]:
_now_mapped_genes.shape

(7120, 3)

In [28]:
_now_mapped_genes.head()

,ENTREZID,ENSEMBL,in_phenomexcan
1,100,ENSG00000196839,True
2,1000,ENSG00000170558,True
3,10000,ENSG00000117020,True
5,10001,ENSG00000133997,True
6,10005,ENSG00000101473,True


In [29]:
_now_mapped_genes_dict = _now_mapped_genes.set_index("ENTREZID").to_dict()["ENSEMBL"]

In [30]:
lincs_data = lincs_data.loc[_now_mapped_genes_dict.keys()].rename(
    index=_now_mapped_genes_dict
)

In [31]:
lincs_data.head()

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


In [32]:
# make sure we have ensembl id only in the index
_tmp = pd.Series(lincs_data.index.map(len)).value_counts()
display(_tmp)
assert _tmp.shape[0] == 1

15    7120
dtype: int64

In [33]:
lincs_data.head()

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


## Remove NaN values

In [34]:
assert not lincs_data.isna().any().any()

## Save

In [35]:
output_file = Path(OUTPUT_RAW_DATA_DIR, "lincs-data.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data/raw/lincs-data.pkl')

In [36]:
lincs_data.to_pickle(output_file)

# Project into MultiPLIER

In [37]:
from multiplier import MultiplierProjection

In [38]:
mproj = MultiplierProjection()

In [39]:
lincs_projection = mproj.transform(lincs_data)

In [40]:
lincs_projection.shape

(987, 1170)

In [41]:
lincs_projection.head()

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
LV1,0.036115,-0.358610,0.091067,0.008923,0.046469,0.136747,0.036151,0.034024,-0.027445,0.060644,...,0.069930,0.021537,0.020511,-0.006250,-0.002660,-0.017210,-0.004029,-0.145490,0.102504,0.057621
LV2,0.012281,0.009738,0.004650,-0.006554,0.021758,0.010425,0.000794,-0.000603,0.011534,-0.004425,...,0.000344,0.001123,-0.012675,-0.004849,0.000375,0.002623,-0.003252,-0.066209,0.025378,0.012453
LV3,-0.005533,-0.059174,0.013454,0.008906,-0.004939,0.033663,-0.010045,0.003824,0.009814,-0.007492,...,-0.021941,-0.002417,0.000155,0.002941,-0.015100,-0.016372,0.012249,0.009216,0.018254,-0.003516
LV4,-0.004151,-0.032884,0.005934,-0.003991,-0.028524,0.002709,0.007001,0.024704,0.005058,-0.004345,...,0.013806,0.003197,0.013099,0.002643,-0.008836,0.023852,0.028957,0.014681,0.000826,-0.009739
LV5,-0.015156,-0.005276,0.025747,0.000346,-0.013932,-0.010911,0.041403,-0.018285,0.003507,-0.007621,...,-0.011577,-0.015956,-0.027614,-0.001707,-0.000542,-0.016918,0.001959,0.024147,0.013004,-0.014166


## Save

In [42]:
output_file = Path(OUTPUT_PROJ_DATA_DIR, "lincs-projection.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/data/proj/lincs-projection.pkl')

In [43]:
lincs_projection.to_pickle(output_file)

# Get reconstructed data

In [44]:
# lincs_data_recon = MultiplierProjection._read_model_z().rename(index=GENE_SYMBOL_TO_ENSEMBL_ID).dot(lincs_projection)

In [45]:
# lincs_data_recon.shape

In [46]:
# lincs_data_recon.head()